<a href="https://colab.research.google.com/github/zjubirdfly/WebCola/blob/master/GeneratePicture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect to google cloud and import data
import uuid
import pandas as pd
import numpy as np
import os
import glob
import shutil
from google.colab import auth
from google.colab import drive
project_id = 'directed-symbol-252319'
bucket_name = "chongqingtaxidata"
raw_data_bucket_name = "raw_taxi_data"
destination_path = "/content/Taxidata/"
auth.authenticate_user()
drive.mount('/content/drive')
!gcloud config set project {project_id}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Updated property [core/project].


In [2]:
#Copy files into system
!mkdir {destination_path}
!gsutil -m cp -r gs://{raw_data_bucket_name}/201703/25 {destination_path}


Copying gs://raw_taxi_data/201703/25/20170325_001.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_002.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_003.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_004.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_005.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_007.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_006.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_008.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_009.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_010.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_011.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_012.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_013.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_014.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_015.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_016.TXT...
Copying gs://raw_taxi_data/201703/25/20170325_017.TXT...
Copying gs://raw_taxi_data/2017

In [0]:
data_cols = ['date','time', 'label', 'tag','lat','lon','speed?','direction?','status', '?']
files = glob.glob("/content/Taxidata/25/20170325_041.TXT")

all_date_data = []
sample_data = []
for file in files:
    sample_data.append(pd.read_csv(file, sep=',',encoding='GB2312', names= data_cols, header=None,  dtype={'time': object}))

all_date_data = pd.concat(sample_data, axis=0, ignore_index=True)

In [15]:
# Generate N*N graph
N = 25

max_lat = max(all_date_data['lat'])
min_lat = min(all_date_data['lat'])
lat_range = (max_lat - min_lat) / N

max_lon = max(all_date_data['lon'])
min_lon = min(all_date_data['lon'])
lon_range = (max_lon - min_lon) / N

graph = np.zeros((N,N))

for index, row in all_date_data.iterrows(): 
  lat = row['lat']
  lon = row['lon']
  #lat index
  lat_index = 0;
  if lat < min_lat:
    lat_index = 0
  elif lat > max_lat:
    lat_index = N - 1
  else:
    lat_index = int((lat - min_lat) / N)

  #lon index
  lon_index = 0;
  if lon < min_lon:
    lon_index = 0
  elif lon > max_lon:
    lon_index = N - 1
  else:
    lon_index = int((lon - min_lon) / N)

  graph[lat_index, lon_index] = graph[lat_index, lon_index] + 1

graph = (255 * graph/graph.mean()).astype(int)
print(graph)

[[   140      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [    13     13     13      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [    39 159154      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0   